<a href="https://colab.research.google.com/github/chandramouli224/python_code/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# -*- coding: utf-8 -*-
"""Untitled4.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1N633elJqKFZEDmRi9Q4oi0OcqPG_0X8v
"""

import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs

class Logistic_Regression:
  def __init__(self, alpha=0.01,precision=1e-07, maxIter=100000):
    self.alpha = alpha
    self.precision = precision
    self.maxIter = maxIter


  def get_unique(self,y):
    return np.unique(y)


  def multivariant_train(self,X,y):
    self.theta = []
    self.cost = []
    X= np.insert(X,0,1,axis=1)
    for i in self.get_unique(y):
      self.y1 = np.where(y == i, 1, 0)
      theta = np.zeros(X.shape[1])
      cost = []
      itr = 0
      theta_deff=theta+1
      while abs(theta_deff).all()>self.precision and itr<self.maxIter:
      #for itr in range(self.maxIter):
        z = X.dot(theta)
        theta_old=theta
        hypothesis = self.sigmoidFunction(z)
        theta, cost1 = self.gradiant_function(X,hypothesis,theta,self.y1)
        theta_deff =theta_old-theta
        cost.append(cost1)
        itr +=1
      self.theta.append((theta, i))
      self.cost.append((cost,i))
    return self
  

  def binary_train(self,X,y):
    X= np.insert(X,0,1,axis=1)
    theta = np.zeros(X.shape[1])
    cost = []
    itr =0
    theta_deff=theta+1
    while theta_deff.all()>self.precision and itr<self.maxIter:
      #for itr in range(self.maxIter):
      z = X.dot(theta)
      theta_old=theta
      hypothesis = self.sigmoidFunction(z)
      theta, cost1 = self.gradiant_function(X,hypothesis,theta,y)
      theta_deff =theta_old-theta
      cost.append(cost1)
      itr +=1
    self.theta = theta
    self.cost = cost
    return self
  
  def train(self, X,y):
    self.m, self.n = X.shape
    self.uni = self.get_unique(y)
    if(len(self.get_unique(y))>2):
      return self.multivariant_train(X,y)
    else:
      return self.binary_train(X,y)
    #return self
    
  def predict(self, X):
    x1=X
    X= np.insert(X,0,1,axis=1)
    X_predicted1 = []
    if(len(self.uni)<=2):
      X_predicted = []
      for i in X:
        if(self.sigmoidFunction(i.dot(self.theta))>0.5):
          X_predicted.append(1)
        else:
          X_predicted.append(0)
      return X_predicted
    else:
      for i in X:
        predict=[]
        for theta, cost in self.theta:
          predict.append((self.sigmoidFunction(i.dot(theta)), cost))
        X_predicted1.append(max(predict)[1])
      return X_predicted1
  

  def sigmoidFunction(self, z):
    return 1/(1 + np.exp(-z))
  
  
  def gradiant_function(self,X,hypothesis,theta,y):
    #gradient = np.dot(X.T, (hypothesis - y)) / m
    gradient = X.T.dot( (hypothesis - y)) / self.m
    #theta -= self.alpha * gradient
    theta =theta - ( self.alpha * gradient )
    cost = (1 / self.m) * (np.sum(-y.T.dot(np.log(hypothesis)) - (1 - y).T.dot(np.log(1 - hypothesis))))
    return theta, cost
  
  def accuracy(self, X, y):
    X=np.array(X)
    y=np.array(y)
    count=0
    for i in range(len(X)):
      if(X[i]==y[i]):
        count +=1
    score = (count / len(y))*100
    #print(X)
    return score
  

  def min_max_scaling(self,X):
    Xnew=pd.DataFrame()
    for col, coldata in X.iteritems():
      xsub=[]
      for x in coldata.values:
        x = (x-min(coldata.values))/(max(coldata.values)-min(coldata.values))
        xsub.append(x)
      Xnew[col]=xsub
    return np.array(Xnew)

def binary_test():
  X, y = make_blobs(n_samples=100, centers=2)
  from sklearn.model_selection import train_test_split
  scores=[]

  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.50)
  logi = Logistic_Regression(precision = 1e-01,  maxIter = 100).train(X_train, y_train)
  predition1 = logi.predict(X_test)
  print((predition1,y_test))
  score1 = logi.accuracy(predition1,y_test)
  print("the accuracy of the model for binary classification is : ",score1)

  from sklearn.linear_model import LogisticRegression
  lg = LogisticRegression().fit(X_train, y_train)
  print("the accuracy of the sklearn model for binary classification is : ",lg.score(X_test,y_test)*100)


def multi_class_Test():
  f = pd.read_csv('beer.txt',delimiter="\t", header=None) 
  f.columns=['calorific_value','nitrogen', 'turbidity', 'style', 'alcohol', 'sugars', 'bitterness', 'beer_id', 'colour', 'degree_of_fermentation']
  overall_accuraccy = []
  overall_sklr=[]
  for _ in range(0,10):
    logi = Logistic_Regression()
    train_data = f.sample(frac=0.667)
    test_data = f.drop(train_data.index)
    X_train = train_data[['calorific_value','nitrogen', 'turbidity', 'alcohol', 'sugars', 'bitterness', 'beer_id', 'colour', 'degree_of_fermentation']]
    X_train = logi.min_max_scaling(X_train)
    y_train = train_data['style']
    X_test = test_data[['calorific_value','nitrogen', 'turbidity', 'alcohol', 'sugars', 'bitterness', 'beer_id', 'colour', 'degree_of_fermentation']]
    X_test = logi.min_max_scaling(X_test)
    y_test = test_data['style']
    model = logi.train(X_train, y_train)
    prediction1 = model.predict(X_test)
    accuracy = logi.accuracy(prediction1,y_test)
    overall_accuraccy.append(accuracy)
    from sklearn.linear_model import LogisticRegression
    from sklearn import preprocessing
    lb_encoder = preprocessing.LabelEncoder()
    y_train=lb_encoder.fit_transform(y_train)
    lg = LogisticRegression(multi_class='ovr').fit(X_train, y_train)
    y_test=lb_encoder.fit_transform(y_test)
    score = lg.score(X_test,y_test)*100
    overall_sklr.append(score)
  print("Accuracy for self implemented logistic regression :", overall_accuraccy)
  print("Accuracy for sklearn logistic regression :", overall_sklr)
  print("##############################################################################################")
  print("avg accuracy for 10 iterations of self inplemented logistic regression model is :", sum(overall_accuraccy)/len(overall_accuraccy))
  print("avg accuracy for 10 iterations of self sklearn logistic regression model is :", sum(overall_sklr)/len(overall_sklr))


multi_class_Test()
binary_test()

ValueError: ignored